# Analiza prometnih nesreč v Sloveniji

In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
#from mapboxgl.viz import *
#from mapboxgl.utils import *

In [4]:
# Import data
prometne_nesrece1 = pd.read_csv('data/novi_pod/pn2005.csv', sep=";",dtype={"SifraCesteNaselja" : str, 
                                                                            "StacionazaDogodka" : str,
                                                                            "SifraOdsekaUlice" : str})

In [5]:
# Import data
prometne_nesrece2 = pd.read_csv('data/novi_pod/pn06-17.csv', sep=";",dtype={"SifraCesteNaselja" : str, 
                                                                            "StacionazaDogodka" : str,
                                                                            "SifraOdsekaUlice" : str})

In [6]:
prometne_nesrece = pd.concat([prometne_nesrece1, prometne_nesrece2])
prometne_nesrece['DatumPN'] = pd.to_datetime(prometne_nesrece['DatumPN'], format='%d.%m.%Y')
prometne_nesrece=prometne_nesrece.sort_values(by=['DatumPN'])
prometne_nesrece['DanVTednu'] = prometne_nesrece['DatumPN'].dt.weekday_name

In [7]:
prometne_nesrece.KlasifikacijaNesrece.unique()

array(['LAŽJA TELESNA POŠKODBA', 'BREZ POŠKODBE', 'HUDA TELESNA POŠKODBA',
       'SMRT'], dtype=object)

In [8]:
udelezenci = pd.read_csv('data/novi_pod/pnose05-17.csv', sep=';', dtype={"UEStalnegaPrebivalisca" : str})

In [9]:
nesrece_po_dnevih = prometne_nesrece.groupby(['DanVTednu'], as_index = False)
dnevi = prometne_nesrece["DanVTednu"].unique()

temp = dict()
dnevi_sum = pd.DataFrame(columns=["DanVTednu", 'BREZ POŠKODBE', 'HUDA TELESNA POŠKODBA', 'LAŽJA TELESNA POŠKODBA', 'SMRT', 'SKUPAJ'])
for dan in dnevi:
    vrste_nesrec = nesrece_po_dnevih.get_group(dan).groupby(["KlasifikacijaNesrece"])
    skupaj = 0
    temp["DanVTednu"] = dan
    for nesreca in vrste_nesrec.groups:
        dolzina = len(vrste_nesrec.groups[nesreca])
        skupaj += dolzina
        temp[nesreca] = dolzina
        
    temp["SKUPAJ"] = skupaj
    dnevi_sum = dnevi_sum.append(temp, ignore_index=True)   

dnevi_sum = dnevi_sum.sort_values(by="SKUPAJ", ascending=[False])

In [10]:
# draws plot
trace1 = go.Bar(
    y=dnevi_sum["DanVTednu"],
    x=dnevi_sum["SMRT"],
    name='Število nesreč s smrtnim izidom',
    orientation = "h",
    marker=dict(
        color='#000000',
    )
    
)
trace2 = go.Bar(
    y=dnevi_sum["DanVTednu"],
    x=dnevi_sum["HUDA TELESNA POŠKODBA"],
    name='Število nesreč s hudo telesno poškodbo',
    orientation = "h",
    marker=dict(
        color='#FF2222',
    )
)
trace3 = go.Bar(
    y=dnevi_sum["DanVTednu"],
    x=dnevi_sum["LAŽJA TELESNA POŠKODBA"],
    name='Število nesreč z lažjo telesno poškodbo',
    orientation = "h",
    marker=dict(
        color='#AAAAAA',
    )
)

trace4 = go.Bar(
    y=dnevi_sum["DanVTednu"],
    x=dnevi_sum["BREZ POŠKODBE"],
    name='Število nesreč brez poškodb',
    orientation = "h",
    marker=dict(
        color='#666666',
    )
)

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    barmode='stack',
    title="Število prometnih nesreč po dnevih v tednu",
    xaxis=dict(
        title="Število prometnih nesreč"
    ),
    yaxis=dict(
        title="Dan v tednu"
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

In [11]:
temp = prometne_nesrece.loc[prometne_nesrece['KlasifikacijaNesrece'] == 'SMRT']
nesrece_po_letih = temp.groupby(temp['DatumPN'].dt.year, as_index = False)

In [12]:
leta = list(nesrece_po_letih.groups.keys())

leta_sum = dict()
for leto, seznam in nesrece_po_letih:
    leta_sum[leto] = len(seznam)

In [13]:
# Create a trace
trace = go.Scatter(
    x = list(leta_sum.keys()),
    y = list(leta_sum.values()),
    marker=dict(
        color='#000000',
    )
)

data = [trace]
layout = go.Layout(
    title="Število smrtnih nesreč skozi leta",
    xaxis=dict(
        title="Leto"
    ),
    yaxis=dict(
        title="Število smrtnih nesreč"
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-line')

In [14]:
x = list(udelezenci['VrednostAlkotesta'])
x = [a for a in x if (0 < a < 4)]

data = [go.Histogram(x=x)]

iplot(data, filename='basic histogram')

In [15]:
vrste_udelezencev = udelezenci.groupby(['VrstaUdelezenca'], as_index = False)
vrste = udelezenci["VrstaUdelezenca"].unique()

pijani = udelezenci.loc[udelezenci['VrednostAlkotesta'] > 0]

In [16]:
vinjeni = udelezenci[(udelezenci["VrednostAlkotesta"] > 0) & (udelezenci["VrednostAlkotesta"] < 4)]
vinjeni.loc[vinjeni['VrednostAlkotesta'] < 0.5, 'VrednostAlkotesta'] = 0.5
vinjeni.loc[(vinjeni['VrednostAlkotesta'] > 0.5) & (vinjeni['VrednostAlkotesta'] < 0.8), 'VrednostAlkotesta'] = 0.8
vinjeni.loc[(vinjeni['VrednostAlkotesta'] > 0.8) & (vinjeni['VrednostAlkotesta'] < 1.5), 'VrednostAlkotesta'] = 1.5
vinjeni.loc[vinjeni['VrednostAlkotesta'] > 1.5, 'VrednostAlkotesta'] = 2
vinjeni;

In [17]:
alko_groups = vinjeni.groupby(['VrstaUdelezenca'], as_index = False)
vrsta_udelezenca = vinjeni["VrstaUdelezenca"].unique()

temp = dict()
alko_sum = pd.DataFrame(columns=['VrstaUdelezenca', 'alko_do_05', 'alko_do_05_do_08', 'alko_do_08_do_15', 'alko_vec_15', 'skupaj'])

print()

for e in vrsta_udelezenca:
    group = alko_groups.get_group(e)["VrednostAlkotesta"]
    
    temp["VrstaUdelezenca"] = e
    temp["alko_do_05"] = len(group[group == 0.5])
    temp["alko_do_05_do_08"] = len(group[group == 0.8])
    temp["alko_do_08_do_15"] = len(group[group == 1.5])
    temp["alko_vec_15"] = len(group[group == 2])
    temp["skupaj"] = len(group[group > 0])
    alko_sum = alko_sum.append(temp, ignore_index=True)   
    
alko_sum = alko_sum.sort_values(by="skupaj", ascending=[False])
alko_sum

,VrstaUdelezenca,alko_do_05,alko_do_05_do_08,alko_do_08_do_15,alko_vec_15,skupaj
0,VOZNIK OSEBNEGA AVTOMOBILA,2786,1536,1349,348,6019
5,VOZNIK TOVORNEGA VOZILA,126,33,49,15,223
2,KOLESAR,66,40,50,13,169
4,VOZNIK KOLESA Z MOTORJEM,68,33,30,2,133
10,VOZNIK MOTORNEGA KOLESA,70,26,11,1,108
1,PEŠEC,26,24,15,6,71
7,VOZNIK TRAKTORJA,30,10,16,5,61
8,POTNIK,20,19,12,1,52
3,VOZNIK KOMBINIRANEGA VOZILA,27,11,9,4,51
11,OSTALO,2,4,6,0,12


In [21]:
"""zmanj=alko_sum.loc[alko_sum['skupaj'] < 10]
zmanj
for v in zmanj.iterrows():
    print(v["alko_do_05 "])"""

'zmanj=alko_sum.loc[alko_sum[\'skupaj\'] < 10]\nzmanj\nfor v in zmanj.iterrows():\n    print(v["alko_do_05 "])'

In [19]:
trace1 = go.Bar(
    x=alko_sum["VrstaUdelezenca"],
    y=alko_sum["alko_do_05"],
    name='manj kot 0,50 g/kg',
    marker=dict(
        color='#bdbdbd',
    )
)
trace2 = go.Bar(
    x=alko_sum["VrstaUdelezenca"],
    y=alko_sum["alko_do_05_do_08"],
    name='od 0,50 do 0,80 g/kg',
    marker=dict(
        color='#9ecae1',
    )
)
trace3 = go.Bar(
    x=alko_sum["VrstaUdelezenca"],
    y=alko_sum["alko_do_08_do_15"],
    name='od 0,80 do 1,50 g/kg',
    marker=dict(
        color='#2b8cbe',
    )
)
trace4 = go.Bar(
    x=alko_sum["VrstaUdelezenca"],
    y=alko_sum["alko_vec_15"],
    name='1,50 g/kg in več',
    marker=dict(
        color='#2b8cbe',
    )
)
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    title="Alkoholiziranost različnih udeležencev prometa",
    barmode='group',
    autosize=False,
    width=950,
    height=650,
    margin=dict(b=200, r=-300),
    xaxis=dict(
        title="Vrsta udeležencev"
    ),
    yaxis=dict(
        title="Število prometnih nesreč"
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')